In [9]:
import numpy as np
from scipy.sparse import *
from skfem import *
from skfem.models.elasticity import linear_elasticity, lame_parameters
from skfem.models.ani_elasticity import *
from skfem.visuals.matplotlib import plot, draw
from skfem.helpers import dot, sym_grad, jump, mul, eye

TETMESH IN THE WILD LIBRARY WITH FORCE Implication

In [15]:
##  TETMESH IN THE WILD LIBRARY INPUT AND TEST ##

import pytetwild
import meshio
import numpy as np
from skfem import *

stl : meshio.Mesh = meshio.read("C:/Users/AlexP/Desktop/BA/STL_Files/TestBenchy.stl")
vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])

cells = meshio.CellBlock('tetra', tetrahedras)
# write intermediate file
meshio.write_points_cells("outFile.vtk", vertices, [cells])

# in case you want to directly continue with it
p = np.array([vertices[:,0],  vertices[:,1],  vertices[:,2]], dtype=np.float64)
t = np.array(tetrahedras, dtype=np.float64).T
m = MeshTet(p, t)



Transforming over 1000 elements to C_CONTIGUOUS.


Starting tetrahedralization process...
Tetrahedralization complete.
Number of vertices: 14061
Number of tetrahedra: 54276
Prepared numpy array for points.
Prepared numpy array for tetrahedra.
Tetrahedralization process completed successfully.


In [21]:
e1 = ElementTetP1()
e = ElementVector(e1)
ib = Basis(m, e, MappingIsoparametric(m, e1), 3)

# Materialparameter
young_modulus = 31.1e9
nu = 0.35

# assemblierung stiffness matrix
K = asm(linear_elasticity(*lame_parameters(young_modulus, nu)), ib)

# label dofs
dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min()))
    }

u = ib.zeros()

# force applied to knots
Kraft = 50000000  
F = np.zeros(u.shape)
up_dofs = dofs['up'].nodal['u^3']
F[up_dofs] = Kraft

# Boundary conditions fix
fixed_dofs = np.hstack([dofs['down'].all()])
u[fixed_dofs] = 0

#set dofs and solve DGL
free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
K_free = K[free_dofs][:, free_dofs]
F_free = F[free_dofs]
u_free = solve(K_free, F_free)
u[free_dofs] = u_free

# calculating upper shift only for the hard coded example
verschiebung_oben = u[up_dofs]


# implement shifts
u[up_dofs] = verschiebung_oben


I = ib.complement_dofs(np.concatenate((dofs['up'], dofs['down'])))
u = solve(*condense(K, x=u, I=I))
print(u[up_dofs])

factor = 50
m_defo = m.translated(factor * u[ib.nodal_dofs])
m_defo.save("C:/Users/AlexP/Desktop/BA/Meshes/Benchynew_" + '.vtk', {"affected": u[ib.nodal_dofs][0]})

[0.07896537 0.07589545 0.07734636 0.07342325 0.07594342 0.06998116
 0.07469312 0.07229413 0.07371766 0.07219656 0.06639942 0.07076339
 0.06737868 0.07010517 0.07017796 0.06648239 0.07729522 0.07708428
 0.06470294 0.06250977 0.07438416 0.07074797 0.0773464  0.06086077
 0.06047857 0.07062286 0.06730954 0.07894504]


FEM IN ALL DIRECTIONS WITHOUT FORCE IMPLECATION 

In [4]:
## FEM IN ALL DIRECTIONS WITHOUT FORCE IMPLECATION ##


import numpy as np
from skfem import *
from skfem.models.elasticity import linear_elasticity, lame_parameters

def fem_in_directions(vertice_shift, mesh_path, young_modulus, nu):
    m = MeshTet()
    m = m.load(mesh_path)

    e1 = ElementTetP1()
    e = ElementVector(e1)
    ib = Basis(m, e, MappingIsoparametric(m, e1), 3)

    K = asm(linear_elasticity(*lame_parameters(young_modulus, nu)), ib)

    dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min()))
    }

    displacements = {
        'left': ('right', 'u^1', -vertice_shift),
        'right': ('left', 'u^1', vertice_shift),
        'up': ('down', 'u^3', vertice_shift),
        'down': ('up', 'u^3', -vertice_shift),
        'front': ('back', 'u^2', vertice_shift),
        'back': ('front', 'u^2', -vertice_shift)
    }

    sf = 10.0

    for direction, (opposite, component, shift) in displacements.items():
        u = ib.zeros()
        u[dofs[direction].nodal[component]] = shift

        # Flatten the DOFs to fix the issue
        I = ib.complement_dofs(np.concatenate([dofs[opposite].all(), dofs[direction].all()]))

        u = solve(*condense(K, x=u, I=I))

        m_shifted = m.translated(sf * u[ib.nodal_dofs])
        m_shifted.save(f"C:/Users/AlexP/Desktop/BA/Meshes/output_{direction}.vtk", {"affected": u[ib.nodal_dofs][0]})

# Example usage
vertice_shift = 1  # or any desired value
mesh_path = "C:/Users/AlexP/Desktop/BA/Meshes/Benchy.msh"
young_modulus = 31.1e9
nu = 0.35

fem_in_directions(vertice_shift, mesh_path, young_modulus, nu)
